In [ ]:
import os
import yaml
from dotenv import load_dotenv
from helperUtil import load_config
from pinecone import Pinecone, ServerlessSpec 
 # Load from .env file
load_dotenv() 
# Read from environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
open_api_key = os.getenv("OPEN_API_KEY")
pinecone_api_key =os.getenv("PINECONE_API_KEY")
grok_api_key = os.getenv("GROK_API_KEY")
# Set for global access
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["OPENAI_API_KEY"] = open_api_key   

 # Load configuration file
config=load_config()
## to do: put them in a class
text_files_data_directory = config["data_directory"]
index_name = config["index_name"]
embedding_model_name= config["embedding_model_name"]
use_lexic_retrieval=config["use_lexic_retrieval"]
use_keyword_in_meta_data=config["use_keyword_in_meta_data"]
re_process_data=config["re_process_data"]
use_query_contextuale=config["re_process_data"]


print(re_process_data)



## init pinecone and create index
pc = Pinecone(api_key=pinecone_api_key)

if index_name  in pc.list_indexes().names():
 pc.delete_index(index_name)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )
index = pc.Index(index_name)





False


In [ ]:
from documents_util import load_documents,chunk_documents,print_data_as_format_json
from db_vector_utils import batch_upload_to_pinecone ,upload_docs_to_pinecone_via_vectorstore
from langchain_pinecone import PineconeVectorStore
from langchain.vectorstores import Pinecone as LangchainPinecone
from sentence_transformers import SentenceTransformer




#cache the calling of Model ~/.cache/huggingface/transformers
from huggingface_hub import hf_hub_download
from langchain.embeddings import HuggingFaceBgeEmbeddings

# Download and cache the config file to get the cache path
cached_model_path = hf_hub_download(repo_id=embedding_model_name, filename="config.json")
print("Model cached at:", cached_model_path)
embedding_model = HuggingFaceBgeEmbeddings(
    model_name=embedding_model_name,
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True},
)





 #Check if the index exists on Pinecone (by name)
if index_name in pc.list_indexes().names() and re_process_data==False:
    print("Index already exists. Connecting...")
    vector_store = LangchainPinecone.from_existing_index(index_name, embedding_model)
else:
    print("Index does not exist. Creating and uploading...")
    print("text_files_data_directory" +text_files_data_directory)
    docs=load_documents(text_files_data_directory)#"./data/AI_engineer"
    #used semantic chunk,and add in meta data contextual data
    docs_after_split=chunk_documents(docs,keywords_chunk=True,summary_chunk=False)
    print(docs_after_split)
    vector_store = upload_docs_to_pinecone_via_vectorstore(
        docs_after_split, embedding_model, index_name, pinecone_api_key
    )
## show_full_pprint: show the data as json view instead of voiewing oit one line
print_data_as_format_json(docs_after_split)

 #normal search by similarity:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 10}
)

# if if use_keyword_in_meta_data = true combined below retriever with lexic retriever with the normal retriever
if use_keyword_in_meta_data:
    from langchain.retrievers import BM25Retriever
    from langchain.retrievers import BM25Retriever
    from langchain.schema import Document

    # Convert your dictionaries to Document objects
    documents = []
    for chunk_dict in docs_after_split:
        doc = Document(
            page_content=chunk_dict["text"],  # BM25 needs this attribute
            metadata=chunk_dict["metadata"]
        )
        documents.append(doc)
    keyword_retriever = BM25Retriever.from_documents(documents)

    ##Note:BM25 + MMR or Similarity → Combine for strongest coverage

    from langchain.retrievers import EnsembleRetriever
    combined_retriever = EnsembleRetriever(
        retrievers=[retriever, keyword_retriever],  # Vector + BM25
        weights=[0.6, 0.4],  # 60% vector similarity, 40% keyword matching
        k=5  # Total results to return
    )


    retriever=combined_retriever


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lb02304/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lb02304/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
No model was supplied, defaulted to distilbert/distilbert-base-cased-distilled-squad and revision 564e9b5 (https://huggingface.co/distilbert/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


Model cached at: C:\Users\lb02304\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2\snapshots\c9745ed1d9f207416be6d2e6f8de32d1f16199bf\config.json


C:\Users\lb02304\AppData\Local\Temp\ipykernel_32736\1476546865.py:18: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(


Index does not exist. Creating and uploading...
text_files_data_directory./data/laracourses/googlellmnotes


📂 Loading files: 100%|██████████| 2/2 [00:12<00:00,  6.34s/it]


[{'id': 'af9c04d0-2427-4673-8729-a6698e11767d', 'chapter': 'Why AI Engineering is the Career of the Future..................................................................8  2', 'source': '0-the AI engineer guide.pdf', 'title': 'TheAIEngineers 3', 'text': "The AI Engineer's Guide: Your Path to Become THE AI ENGINEER!..................................... 3 What You'll Find in This Ebook................................................................................................ 4  Introduction..............................................................................................................................5  1. Why AI Engineering is the Career of the Future..................................................................8  2", 'metadata': {'summary': '', 'prev_summary': None, 'chunk_index': 0, 'chunk_length': 491, 'prev_keywords': None, 'keywords': ['ai', 'engineer', 'guide', 'path', 'become']}}, {'id': '8d3ad61b-340d-446a-8b52-91a0a9c2d719', 'chapter': 'Roadmap to AI Su

In [ ]:

import json
import os
import gradio as gr
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.memory.chat_message_histories import FileChatMessageHistory
from langchain.memory import ConversationBufferMemory
from langsmith import traceable
from langchain_community.callbacks import get_openai_callback
from sklearn.metrics.pairwise import cosine_similarity
from langchain.schema import HumanMessage, AIMessage


from chatutils import compress_chat_history, is_greeting_or_compliment , random_greeting_response

# --------------- Prompt & Model Setup ---------------

with open("AIprompts.json", "r",  encoding="utf-8") as file:
    data = json.load(file)
    contextualize_system_prompt = data["contextualize_system_prompt"]
    system_prompt = data["system_prompt"]
    


llm = ChatGroq(
    groq_api_key=grok_api_key,
    model_name='llama3-70b-8192',
    temperature=0.1
)
## if i need to use query + context to re-formulate my query
if use_query_contextuale:
    contextualize_prompt = ChatPromptTemplate.from_messages([
        ("system", contextualize_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}")
    ])
    history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_prompt)


my_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])


question_answer_chain = create_stuff_documents_chain(llm, my_prompt)

# --------------- Memory Setup ---------------

chat_memory = FileChatMessageHistory("chat_logs/session1.json")
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    chat_memory=chat_memory
)



# --------------- Chat Handler ---------------

@traceable(name="chat_rag_session")
def chat_with_model(history, new_message, chat_history):
    #  Apply compression to chat history
    trimmed_history = compress_chat_history(chat_history, max_turns=5, max_length=300)

    # Handle greetings or simple questions
    if  is_greeting_or_compliment(new_message):
            result = random_greeting_response()
            history.append((new_message, result))
            chat_history.extend([
                    HumanMessage(content=new_message),
                    AIMessage(content=result)
            ])
            return history, ""

    else:
        modified_query = new_message
       # in case we need to rephrase the input query again
        if use_query_contextuale:
          print(" Using context-aware query reformulation...")

          retrieved_docs = history_aware_retriever.invoke({
         "input": new_message,
         "chat_history": trimmed_history
         })
          used_context = True
       # proceed with retrieve the document
        else:
         retrieved_docs = retriever.get_relevant_documents(modified_query)
         used_context = False# since we used the "query as is" so log this info
         #score=0

        #get_openai_callback() is a context manager provided by LangChain even of not using  openai llm
        # tracks the OpenAI API usage during everything inside the block. 
        with get_openai_callback() as cb:
            result = question_answer_chain.invoke({
                "input": new_message,
                "chat_history": trimmed_history,
                "context": retrieved_docs
            })

            token_usage = {
                "total_tokens": cb.total_tokens,
                "prompt_tokens": cb.prompt_tokens,
                "completion_tokens": cb.completion_tokens
            }

        # Debug info
        payload_debug = {
            "input": new_message,
            "contextualized_query": modified_query,
            #"similarity_score": float(score),
            "used_contextualized": bool(used_context),
            "chat_history": [msg.content for msg in trimmed_history],
            "top_k_docs": [doc.page_content[:300] for doc in retrieved_docs],
            "final_llm_output": result,
            "token_usage": token_usage
        }

        print("\n📤 Full Debug Trace (LLM Payload):")
        print(json.dumps(payload_debug, indent=2))

        os.makedirs("logs", exist_ok=True)
        with open("logs/debug_trace.txt", "a", encoding="utf-8") as log_file:
            log_file.write(json.dumps(payload_debug, indent=2) + "\n\n")

        bot_reply = result

        # Update state
        chat_history.extend([
            HumanMessage(content=new_message),
            AIMessage(content=bot_reply)
        ])
        history.append((new_message, bot_reply))

        return history, ""
# --------------- Gradio UI ---------------

###
custom_css = """
    .message {
        font-weight: bold;
        font-size: 16px;
    }
    """
def gradio_chat_app():
    with gr.Blocks(css=custom_css) as app:
     
        chat_history = gr.State([])
        gr.Markdown("# Ask AI Skool")
        gr.Markdown("Chat with AI... and it will reply to your technical inquiries.")

        chatbot = gr.Chatbot(label="Chat Interface")
        user_input = gr.Textbox(label="Your message", placeholder="Type something...", lines=1)
        send_button = gr.Button("Send")
        clear_button = gr.Button("Clear chat")

        def clear_chat():
            return [], "", []

        send_button.click(
            fn=chat_with_model,
            inputs=[chatbot, user_input, chat_history],
            outputs=[chatbot, user_input]
        )

        user_input.submit(
            fn=chat_with_model,
            inputs=[chatbot, user_input, chat_history],
            outputs=[chatbot, user_input]
        )

        clear_button.click(
            fn=clear_chat,
            inputs=[],
            outputs=[chatbot, user_input, chat_history]
        )

    return app

# --------------- Launch ---------------

if __name__ == "__main__":
    app = gradio_chat_app()
    app.launch(share=True)


C:\Users\lb02304\AppData\Local\Temp\ipykernel_32736\534536238.py:57: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(
d:\AI codes\env\Lib\site-packages\gradio\components\chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 7860): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 7861): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normal

* Running on local URL:  http://127.0.0.1:7862
* Running on public URL: https://377c73497053e42613.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


C:\Users\lb02304\AppData\Local\Temp\ipykernel_32736\534536238.py:93: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(modified_query)



📤 Full Debug Trace (LLM Payload):
{
  "input": "i need a roadmap for python",
  "contextualized_query": "i need a roadmap for python",
  "similarity_score": 0.0,
  "used_contextualized": false,
  "chat_history": [],
  "top_k_docs": [
    "\u2022\nCode Cleaning and Abstraction: Writing clean code and understanding abstraction (like writing functions to avoid repeating code) is important, especially when working with large codebases.\n\u2022\nCode Efficiency: Understanding which data structure or method to use for large datasets (e.g., Lists vs. Di",
    "2.2 Programming Fundamentals (Python) Python is a fundamental skill required for AI.\n\u2022\nData Types, Loops, Functions: Basic programming concepts like data types, loops, and functions are essential. Loops, for instance, are crucial for iterating through data, especially with large datasets. There are ",
    ". The program emphasizes benefiting from resources that might not be taught extensively in schools or universities. It cover

#enhancing the context query

##ANUAL STESP BY AI